In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

def extract_dominant_colors(image_path, k=5):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(image)
    
    dominant_colors = kmeans.cluster_centers_.astype(int)
    _, counts = np.unique(kmeans.labels_, return_counts=True)
    percentages = counts / counts.sum()
    
    return dominant_colors, percentages

def calculate_min_distance(colors1, colors2):
    distances = []
    for color1 in colors1:
        for color2 in colors2:
            dist = np.linalg.norm(np.array(color1) - np.array(color2))
            distances.append(dist)
    return distances

def calculate_weighted_similarity(colors1, colors2, percentages1, percentages2):
    distances = []
    for i, color1 in enumerate(colors1):
        for j, color2 in enumerate(colors2):
            dist = np.linalg.norm(np.array(color1) - np.array(color2))
            weighted_distance = dist * (percentages1[i] + percentages2[j]) / 2  # Average percentage weight
            distances.append(weighted_distance)
    return np.mean(distances)

def process_images(image_folder, k=5):
    dominant_colors_rgb_list = []
    percentages_list = []
    image_names = []

    for i in range(1, 8):
        image_path = os.path.join(image_folder, f'im{i}.jpeg')
        print(f"Processing {image_path}...")
        
        dominant_colors_rgb, percentages = extract_dominant_colors(image_path, k)
        
        dominant_colors_rgb_list.append(dominant_colors_rgb)
        percentages_list.append(percentages)
        image_names.append(f'Image {i}')
    
    min_distances = []
    
    print("\nMinimum Distances between images:")
    for i in range(len(dominant_colors_rgb_list)):
        for j in range(i + 1, len(dominant_colors_rgb_list)):
            distances = calculate_min_distance(dominant_colors_rgb_list[i], dominant_colors_rgb_list[j])
            min_distance = np.min(distances)
            min_distances.append((image_names[i], image_names[j], min_distance))
            print(f"Min Distance between {image_names[i]} and {image_names[j]}: {min_distance:.4f}")

    print("\nSimilarity based on minimum distances between images:")
    for (img1, img2, min_distance) in min_distances:
        # Calculate similarity using weighted distances
        idx1 = image_names.index(img1)
        idx2 = image_names.index(img2)
        weighted_similarity = calculate_weighted_similarity(dominant_colors_rgb_list[idx1], dominant_colors_rgb_list[idx2],
                                                            percentages_list[idx1], percentages_list[idx2])
        similarity = 1 / (1 + weighted_similarity)  # Adjusted similarity calculation
        print(f"Similarity between {img1} and {img2}: {similarity:.4f}")

def visualize_colors(dominant_colors_rgb_list):
    plt.figure(figsize=(10, 5))
    for i, colors in enumerate(dominant_colors_rgb_list):
        plt.subplot(1, len(dominant_colors_rgb_list), i + 1)
        plt.imshow([colors])
        plt.axis('off')
        plt.title(f'Image {i + 1}')
    plt.show()

# Example usage
image_folder = '/Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images'  # Update this path to your image folder
process_images(image_folder)

Processing /Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images/im1.jpeg...
Processing /Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images/im2.jpeg...
Processing /Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images/im3.jpeg...
Processing /Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images/im4.jpeg...
Processing /Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images/im5.jpeg...
Processing /Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images/im6.jpeg...
Processing /Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab3/images/im7.jpeg...

Minimum Distances between images:
Min Distance between Image 1 and Image 2: 17.8326
Min Distance between Image 1 and Image 3: 59.8164
Min Distance between Image 1 and Image 4: 34.3802
Min Distan